In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import optimizers,losses,layers
import pickle
from glob import glob
from os.path import basename
import networkx as nx
import random

In [2]:
dataset = 'IMDB' #LINUX

In [3]:
def load_generated_graphs(dataset_name, file_name='generated_graph_500'):

    dir = './dataset/' + dataset_name + '/' + file_name
    g = open(dir, 'rb')
    generated_graphs = pickle.load(g)
    g.close()
    return generated_graphs

training_pairs = load_generated_graphs(dataset, file_name='generated_graph_pairs')
test_gt_ged = load_generated_graphs(dataset, file_name='ged_matrix_test')

In [4]:
def sorted_nicely(l):
    def tryint(s):
        try:
            return int(s)
        except:
            return s

    import re
    def alphanum_key(s):
        return [tryint(c) for c in re.split('([0-9]+)', s)]

    return sorted(l, key=alphanum_key)

def load_graphs(dataset_name, train_or_test='train'):
    graphs = []
    dir = './dataset/' + dataset_name + '/' + train_or_test
    for file in sorted_nicely(glob(dir + '/*.gexf')):
        gid = int(basename(file).split('.')[0])
        g = nx.read_gexf(file)
        g.graph['gid'] = gid
        graphs.append(g)
        if not nx.is_connected(g):
            raise RuntimeError('{} not connected'.format(gid))
    return graphs

training_graphs = load_graphs(dataset, train_or_test='train')
testing_graphs = load_graphs(dataset, train_or_test='test')

In [5]:
class MyModel(tf.keras.Model):

    def __init__(self):
        super().__init__()
    
        self.input_in = tf.keras.Input(shape=(16,))
        self.fully_connected_first_in = tf.keras.layers.Dense(16, activation="relu")
        self.fully_connected_second_in = tf.keras.layers.Dense(8, activation="relu")
        self.fully_connected_third_in = tf.keras.layers.Dense(4, activation="relu")
        self.scoring_layer_in = tf.keras.layers.Dense(1, activation="sigmoid")

        self.scoring_layer_in = keras.Sequential([
            self.input_in, ##feature_count = tensor_neurons = 16
            self.fully_connected_first_in,
            self.fully_connected_second_in,
            self.fully_connected_third_in,
            self.scoring_layer_in])

        
        self.input_ie = tf.keras.Input(shape=(16,))
        self.fully_connected_first_ie = tf.keras.layers.Dense(16, activation="relu")
        self.fully_connected_second_ie = tf.keras.layers.Dense(8, activation="relu")
        self.fully_connected_third_ie = tf.keras.layers.Dense(4, activation="relu")
        self.scoring_layer_ie = tf.keras.layers.Dense(1, activation="sigmoid")

        self.scoring_layer_ie = keras.Sequential([
            self.input_ie, ##feature_count = tensor_neurons = 16
            self.fully_connected_first_ie,
            self.fully_connected_second_ie,
            self.fully_connected_third_ie,
            self.scoring_layer_ie])

        number_of_labels = 1
        self.input_dim_in = 11
        #7 for linux; 11 for IMDB
        self.input_dim_ie = 60
        # 21 for linux; 60 for IMDB 

        initializer1 = tf.keras.initializers.GlorotUniform()
        self.weight_matrix_in = initializer1(shape = (self.input_dim_in, self.input_dim_in, 16), dtype = tf.float32)
        self.weight_matrix_block_in = initializer1(shape= (16, 2*self.input_dim_in), dtype = tf.float32)
        self.bias_in = initializer1(shape = (16, 1), dtype = tf.float32)
        
        initializer2 = tf.keras.initializers.GlorotUniform()
        self.weight_matrix_ie = initializer2(shape = (self.input_dim_ie, self.input_dim_ie, 16), dtype = tf.float32)
        self.weight_matrix_block_ie = initializer2(shape= (16, 2*self.input_dim_ie), dtype = tf.float32)
        self.bias_ie = initializer2(shape = (16, 1), dtype = tf.float32)


    def call(self, data):
    
        adj_1 = tf.constant(np.array(data["edge_index_1"].todense()),dtype=tf.float32)
        adj_2 = tf.constant(np.array(data["edge_index_2"].todense()),dtype=tf.float32)
        features_1, features_2 = data["features_1"], data["features_2"]

        
        #GAL 
        graph1_hidden1 = tf.matmul(adj_1,features_1)
        graph1_hidden2 = tf.matmul(adj_1,graph1_hidden1)

        graph2_hidden1 = tf.matmul(adj_2,features_2)
        graph2_hidden2 = tf.matmul(adj_2,graph2_hidden1)
        
        num1 = 10
        # 10 for imdb; 6 for linux 
        num2 = 50
        # 50 for imdb; 15 for linux
        Graph1_hidden1, Graph1_hidden2, Graph2_hidden1, Graph2_hidden2 = [], [], [], []
        for i in range(graph1_hidden1.shape[0]):
            if(graph1_hidden1[i][0] >= num1):
                Graph1_hidden1.append([0.0]*(num1-1) + [1.0])
            else:
                Graph1_hidden1.append([1.0 if graph1_hidden1[i][0] == j else 0.0 for j in range(num1)])

            if(graph1_hidden2[i][0] >= num2):
                Graph1_hidden2.append([0.0]*(num2-1) + [1.0])
            else:
                Graph1_hidden2.append([1.0 if graph1_hidden2[i][0] == j else 0.0 for j in range(num2)])

        for i in range(graph2_hidden1.shape[0]):
            if(graph2_hidden1[i][0] >= num1):
                Graph2_hidden1.append([0.0]*(num1-1) + [1.0])
            else:
                Graph2_hidden1.append([1.0 if graph2_hidden1[i][0] == j else 0.0 for j in range(num1)])

            if(graph2_hidden2[i][0] >= num2):
                Graph2_hidden2.append([0.0]*(num2-1) + [1.0])
            else:
                Graph2_hidden2.append([1.0 if graph2_hidden2[i][0] == j else 0.0 for j in range(num2)])
            
        Graph1_hidden1 =  tf.constant(np.array(Graph1_hidden1),dtype = tf.float32)
        Graph1_hidden2 = tf.constant(np.array(Graph1_hidden2),dtype = tf.float32)
        Graph2_hidden1 = tf.constant(np.array(Graph2_hidden1),dtype = tf.float32)
        Graph2_hidden2 = tf.constant(np.array(Graph2_hidden2),dtype = tf.float32)


        graph1_01concat = tf.concat([features_1, Graph1_hidden1], axis=1)
        graph2_01concat = tf.concat([features_2, Graph2_hidden1], axis=1)
        graph1_12concat = tf.concat([Graph1_hidden1, Graph1_hidden2], axis=1)
        graph2_12concat = tf.concat([Graph2_hidden1, Graph2_hidden2], axis=1)

        graph1_01pooled = tf.expand_dims(tf.reduce_sum(graph1_01concat,axis=0),1)
        graph1_12pooled = tf.expand_dims(tf.reduce_sum(graph1_12concat,axis=0),1)
        graph2_01pooled = tf.expand_dims(tf.reduce_sum(graph2_01concat,axis=0),1)
        graph2_12pooled = tf.expand_dims(tf.reduce_sum(graph2_12concat,axis=0),1)
        


        scoring = tf.matmul(tf.transpose(graph1_01pooled, perm=[1,0]), self.weight_matrix_in.reshape(self.input_dim_in, -1))
        scoring = scoring.reshape(self.input_dim_in, 16)
        scoring = tf.matmul(tf.transpose(scoring, perm=[1,0]), graph2_01pooled)
        combined_representation = tf.concat((graph1_01pooled, graph2_01pooled),axis=0)
        block_scoring = tf.matmul(self.weight_matrix_block_in, combined_representation)
        scores_in = tf.nn.relu(scoring + block_scoring + self.bias_in)
        scores_in = tf.transpose(scores_in, perm=[1,0])


        scores_in = self.scoring_layer_in(scores_in)
        #print("score_in: \n", scores_in)


        scoring = tf.matmul(tf.transpose(graph1_12pooled, perm=[1,0]), self.weight_matrix_ie.reshape(self.input_dim_ie, -1))
        scoring = scoring.reshape(self.input_dim_ie, 16)
        scoring = tf.matmul(tf.transpose(scoring, perm=[1,0]), graph2_12pooled)
        combined_representation = tf.concat((graph1_12pooled, graph2_12pooled),axis = 0)
        block_scoring = tf.matmul(self.weight_matrix_block_ie, combined_representation)
        scores_ie = tf.nn.relu(scoring + block_scoring + self.bias_ie)
        scores_ie = tf.transpose(scores_ie, perm=[1,0])


        scores_ie = self.scoring_layer_ie(scores_ie)
        #print("score_ie: \n", scores_ie)
        
        return(tf.concat([scores_in, scores_ie], axis=1))

In [6]:
from tensorflow.python.ops.numpy_ops import np_config
np_config.enable_numpy_behavior()
def transfer_to_tensorflow(data, type_specified=True):

        new_data = dict()
        graph1, graph2 = data['graph_pair'][0], data['graph_pair'][1]
        nodes1, nodes2 = list(graph1.nodes()), list(graph2.nodes())

        features_1, features_2 = [], []

        for n in graph1.nodes():
            features_1.append([1.0])

        for n in graph2.nodes():
            features_2.append([1.0])

        features_1, features_2 = tf.constant(np.array(features_1),dtype = tf.float32), tf.constant(np.array(features_2),dtype = tf.float32)

        new_data["edge_index_1"], new_data["edge_index_2"] = nx.adjacency_matrix(graph1), nx.adjacency_matrix(graph2)
        new_data["features_1"], new_data["features_2"] = features_1, features_2
        
        if(type_specified):
            norm_ged = [data['ged'][key] / (0.5 * (graph1.number_of_nodes() + graph2.number_of_nodes())) for key in ['in', 'ie']]
            norm_ged = np.array(norm_ged)
            new_data["target"] = tf.cast(tf.convert_to_tensor(np.exp(-norm_ged)).reshape(1, -1), tf.float32)
            
            norm_gt_ged = (data['ged']['in'] + data['ged']['ie']) / (0.5 * (graph1.number_of_nodes() + graph2.number_of_nodes()))
            new_data["gt_ged"] = tf.cast(tf.convert_to_tensor(np.exp(-norm_gt_ged).reshape(1, 1)).reshape(1, -1),tf.float32)
        else:
            norm_gt_ged = data['ged'] / (0.5 * (graph1.number_of_nodes() + graph2.number_of_nodes()))
            new_data["gt_ged"] = tf.cast(tf.convert_to_tensor(np.exp(-norm_gt_ged).reshape(1, 1)).reshape(1, -1),tf.float32)

        return new_data

In [7]:
batch_size = 128
epochs = 1
mse = tf.keras.losses.MeanSquaredError()
optimizer = keras.optimizers.Adam(learning_rate = 1e-3)
model = MyModel()

for epoch in range(epochs):
    random.shuffle(training_pairs)
    batches = []
    for graph in range(0, len(training_pairs), batch_size):
          batches.append(training_pairs[graph:graph+batch_size])
    iteration = 0
    for batch in batches:
        losses = 0  
        with tf.GradientTape() as tape:
            for graph_pair in batch:
                data = transfer_to_tensorflow(graph_pair)
                model.compile(optimizer=optimizer, loss=mse)
                test = model(data)
                losses += mse(data["target"], test)

        grads = tape.gradient(losses, model.trainable_variables)
        optimizer.apply_gradients(zip(grads, model.trainable_variables))

        loss = losses.numpy()    
        print ("Iteration: ", iteration, "loss: ", loss/len(batch))  
        iteration+=1
        
        
test_scores = []
for graph_1 in testing_graphs:
    for graph_2 in training_graphs:
        if((graph_1.graph['gid'], graph_2.graph['gid']) in test_gt_ged):
            curr_graph_pair = {'graph_pair': [graph_1, graph_2], 'ged':test_gt_ged[(graph_1.graph['gid'], graph_2.graph['gid'])]}
            data = transfer_to_tensorflow(curr_graph_pair, type_specified=False)
            prediction = model(data,training=False)
            prediction = tf.math.exp(tf.math.reduce_sum(tf.math.log(prediction))).reshape(1, -1)
            current_error = mse(prediction, data["gt_ged"])
            test_scores.append(current_error)


    for graph_2 in testing_graphs:
        if((graph_1.graph['gid'], graph_2.graph['gid']) in test_gt_ged):
            curr_graph_pair = {'graph_pair': [graph_1, graph_2], 'ged':test_gt_ged[(graph_1.graph['gid'], graph_2.graph['gid'])]}
            data = transfer_to_tensorflow(curr_graph_pair, type_specified=False)
            prediction = model(data,training=False)
            prediction = tf.math.exp(tf.math.reduce_sum(tf.math.log(prediction))).reshape(1, -1)
            current_error = mse(prediction, data["gt_ged"])
            test_scores.append(current_error)

model_error = sum(test_scores) / len(test_scores)
print("\nModel test error: " + str(model_error))

C:\Users\huang\AppData\Local\Temp\ipykernel_3464\1718527493.py:19: FutureWarning: adjacency_matrix will return a scipy.sparse array instead of a matrix in Networkx 3.0.
  new_data["edge_index_1"], new_data["edge_index_2"] = nx.adjacency_matrix(graph1), nx.adjacency_matrix(graph2)


Iteration:  0 loss:  0.1474183350801468
Iteration:  1 loss:  0.12406615167856216
Iteration:  2 loss:  0.14257702231407166
Iteration:  3 loss:  0.13952940702438354
Iteration:  4 loss:  0.1331179141998291
Iteration:  5 loss:  0.11901863664388657
Iteration:  6 loss:  0.1162961944937706
Iteration:  7 loss:  0.12788747251033783
Iteration:  8 loss:  0.12176379561424255
Iteration:  9 loss:  0.1034935787320137
Iteration:  10 loss:  0.11193995177745819
Iteration:  11 loss:  0.0968874990940094
Iteration:  12 loss:  0.10028363019227982
Iteration:  13 loss:  0.09530045092105865
Iteration:  14 loss:  0.09575287997722626
Iteration:  15 loss:  0.1052587702870369
Iteration:  16 loss:  0.09970445930957794
Iteration:  17 loss:  0.09433458000421524
Iteration:  18 loss:  0.08891436457633972
Iteration:  19 loss:  0.09065637737512589
Iteration:  20 loss:  0.09464081376791
Iteration:  21 loss:  0.08706169575452805
Iteration:  22 loss:  0.07632549852132797
Iteration:  23 loss:  0.08062667399644852
Iteration: 

Iteration:  193 loss:  0.04341807961463928
Iteration:  194 loss:  0.041945844888687134
Iteration:  195 loss:  0.04489196091890335
Iteration:  196 loss:  0.048452228307724
Iteration:  197 loss:  0.04321691393852234
Iteration:  198 loss:  0.048492107540369034
Iteration:  199 loss:  0.04147171229124069
Iteration:  200 loss:  0.041442424058914185
Iteration:  201 loss:  0.04065999761223793
Iteration:  202 loss:  0.04438045993447304
Iteration:  203 loss:  0.04306057468056679
Iteration:  204 loss:  0.043831098824739456
Iteration:  205 loss:  0.04430954158306122
Iteration:  206 loss:  0.0447230339050293
Iteration:  207 loss:  0.04410748928785324
Iteration:  208 loss:  0.04784536361694336
Iteration:  209 loss:  0.04389311373233795
Iteration:  210 loss:  0.04283349588513374
Iteration:  211 loss:  0.042974986135959625
Iteration:  212 loss:  0.04386138543486595
Iteration:  213 loss:  0.0446229912340641
Iteration:  214 loss:  0.045887719839811325
Iteration:  215 loss:  0.04520284757018089
Iteration

Iteration:  383 loss:  0.03404703736305237
Iteration:  384 loss:  0.03427949547767639
Iteration:  385 loss:  0.03570014610886574
Iteration:  386 loss:  0.031943440437316895
Iteration:  387 loss:  0.034232985228300095
Iteration:  388 loss:  0.035765063017606735
Iteration:  389 loss:  0.03244692087173462
Iteration:  390 loss:  0.03150935098528862
Iteration:  391 loss:  0.034244194626808167
Iteration:  392 loss:  0.03523765504360199
Iteration:  393 loss:  0.03514372184872627
Iteration:  394 loss:  0.03495832160115242
Iteration:  395 loss:  0.03129163756966591
Iteration:  396 loss:  0.032546598464250565
Iteration:  397 loss:  0.03240533545613289
Iteration:  398 loss:  0.029195422306656837
Iteration:  399 loss:  0.0300351120531559
Iteration:  400 loss:  0.031222552061080933
Iteration:  401 loss:  0.03231372311711311
Iteration:  402 loss:  0.031127769500017166
Iteration:  403 loss:  0.03106892853975296
Iteration:  404 loss:  0.03460708260536194
Iteration:  405 loss:  0.03332042321562767
Iter

Iteration:  572 loss:  0.023954220116138458
Iteration:  573 loss:  0.025342902168631554
Iteration:  574 loss:  0.027669351547956467
Iteration:  575 loss:  0.027327213436365128
Iteration:  576 loss:  0.02909797430038452
Iteration:  577 loss:  0.02413080632686615
Iteration:  578 loss:  0.027270175516605377
Iteration:  579 loss:  0.027697401121258736
Iteration:  580 loss:  0.025715524330735207
Iteration:  581 loss:  0.025949543341994286
Iteration:  582 loss:  0.026387261226773262
Iteration:  583 loss:  0.02531997114419937
Iteration:  584 loss:  0.024073401466012
Iteration:  585 loss:  0.024164196103811264
Iteration:  586 loss:  0.02390456758439541
Iteration:  587 loss:  0.024942144751548767
Iteration:  588 loss:  0.028329389169812202
Iteration:  589 loss:  0.024884622544050217
Iteration:  590 loss:  0.02613304741680622
Iteration:  591 loss:  0.02509273588657379
Iteration:  592 loss:  0.02608315460383892
Iteration:  593 loss:  0.026882685720920563
Iteration:  594 loss:  0.02609515562653541

...

Iteration:  2778 loss:  0.006784592755138874
Iteration:  2779 loss:  0.007207635324448347
Iteration:  2780 loss:  0.006708287633955479
Iteration:  2781 loss:  0.0063385250978171825
Iteration:  2782 loss:  0.005949263460934162
Iteration:  2783 loss:  0.006771598942577839
Iteration:  2784 loss:  0.006845938507467508
Iteration:  2785 loss:  0.0077558644115924835
Iteration:  2786 loss:  0.006657431833446026
Iteration:  2787 loss:  0.007142743095755577
Iteration:  2788 loss:  0.006960808765143156
Iteration:  2789 loss:  0.006966552697122097
Iteration:  2790 loss:  0.007014227565377951
Iteration:  2791 loss:  0.0065483711659908295
Iteration:  2792 loss:  0.007905750535428524
Iteration:  2793 loss:  0.006670552771538496
Iteration:  2794 loss:  0.007095664273947477
Iteration:  2795 loss:  0.0073994058184325695
Iteration:  2796 loss:  0.007606278173625469
Iteration:  2797 loss:  0.006843813695013523
Iteration:  2798 loss:  0.0069495197385549545
Iteration:  2799 loss:  0.007664615288376808
Itera

Iteration:  2960 loss:  0.006188014522194862
Iteration:  2961 loss:  0.007059782277792692
Iteration:  2962 loss:  0.007473643869161606
Iteration:  2963 loss:  0.006582244765013456
Iteration:  2964 loss:  0.006685473956167698
Iteration:  2965 loss:  0.006481609307229519
Iteration:  2966 loss:  0.007765378803014755
Iteration:  2967 loss:  0.006659686099737883
Iteration:  2968 loss:  0.005587632302194834
Iteration:  2969 loss:  0.00701878359541297
Iteration:  2970 loss:  0.007246016059070826
Iteration:  2971 loss:  0.005891719367355108
Iteration:  2972 loss:  0.006830660626292229
Iteration:  2973 loss:  0.0062738945707678795
Iteration:  2974 loss:  0.007245358545333147
Iteration:  2975 loss:  0.005678987130522728
Iteration:  2976 loss:  0.006196212489157915
Iteration:  2977 loss:  0.006890450604259968
Iteration:  2978 loss:  0.0067514474503695965
Iteration:  2979 loss:  0.0070964619517326355
Iteration:  2980 loss:  0.006895769387483597
Iteration:  2981 loss:  0.006181623786687851
Iteratio

Iteration:  3143 loss:  0.0060861194506287575
Iteration:  3144 loss:  0.00618546549230814
Iteration:  3145 loss:  0.0076319011859595776
Iteration:  3146 loss:  0.006925827823579311
Iteration:  3147 loss:  0.006824163254350424
Iteration:  3148 loss:  0.006363609805703163
Iteration:  3149 loss:  0.007861348800361156
Iteration:  3150 loss:  0.0064854491502046585
Iteration:  3151 loss:  0.00669375853613019
Iteration:  3152 loss:  0.006744214799255133
Iteration:  3153 loss:  0.006687296088784933
Iteration:  3154 loss:  0.006615473888814449
Iteration:  3155 loss:  0.006277687381953001
Iteration:  3156 loss:  0.006677531637251377
Iteration:  3157 loss:  0.007335582748055458
Iteration:  3158 loss:  0.0057546216994524
Iteration:  3159 loss:  0.006081087049096823
Iteration:  3160 loss:  0.006048111245036125
Iteration:  3161 loss:  0.006086212582886219
Iteration:  3162 loss:  0.006540733855217695
Iteration:  3163 loss:  0.006405166815966368
Iteration:  3164 loss:  0.005720154382288456
Iteration: 

Iteration:  3325 loss:  0.006033202167600393
Iteration:  3326 loss:  0.0065581765957176685
Iteration:  3327 loss:  0.006548264063894749
Iteration:  3328 loss:  0.0068865204229950905
Iteration:  3329 loss:  0.006632160861045122
Iteration:  3330 loss:  0.006877765990793705
Iteration:  3331 loss:  0.007761091459542513
Iteration:  3332 loss:  0.007347635924816132
Iteration:  3333 loss:  0.006640521343797445
Iteration:  3334 loss:  0.006204883102327585
Iteration:  3335 loss:  0.006196048576384783
Iteration:  3336 loss:  0.008226937614381313
Iteration:  3337 loss:  0.006730525754392147
Iteration:  3338 loss:  0.006631852593272924
Iteration:  3339 loss:  0.007265287917107344
Iteration:  3340 loss:  0.007081886287778616
Iteration:  3341 loss:  0.0065387520007789135
Iteration:  3342 loss:  0.006359930615872145
Iteration:  3343 loss:  0.0071937614120543
Iteration:  3344 loss:  0.006566227413713932
Iteration:  3345 loss:  0.006275573745369911
Iteration:  3346 loss:  0.005693236831575632
Iteration

KeyboardInterrupt: 